In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import bentoml
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv('../data/brutos/resultado_pocos.csv', sep=';', encoding='latin-1')
df.columns = df.columns.str.strip()

# 2. Gera dados sintéticos
def gerar_dados_sinteticos(df_original):
    np.random.seed(42)
    n = len(df_original)

    df_original['porosidade_pct'] = np.clip(np.random.normal(18, 4, n), 8, 28)
    df_original['espessura_zona_prod_m'] = np.random.uniform(5, 60, n)
    df_original['pressao_reservatorio_psi'] = np.random.lognormal(mean=8, sigma=0.6, size=n)
    df_original['pressao_reservatorio_psi'] = df_original['pressao_reservatorio_psi'].clip(100, 5000)
    df_original['saturacao_oleo_pct'] = np.random.uniform(30, 85, n)

    litologias = ['Arenito', 'Folhelho', 'Calcário', 'Conglomerado']
    probs = [0.6, 0.25, 0.1, 0.05]
    df_original['litologia'] = np.random.choice(litologias, size=n, p=probs)

    df_original['produtividade_inicial_bpd'] = (
        0.8 * df_original['porosidade_pct'] +
        1.2 * df_original['espessura_zona_prod_m'] +
        0.015 * df_original['pressao_reservatorio_psi'] +
        np.random.normal(0, 20, n)
    )

    df_original['produtividade_inicial_bpd'] = df_original['produtividade_inicial_bpd'].clip(lower=50)
    return df_original

df_sintetico = gerar_dados_sinteticos(df.copy())

In [15]:
df

,Nome Poço ANP,Nome Poço Operador,Bloco,Bacia,Campo,Contrato,Operador,Início Perfuração,Lâmina d'água (m),Latitude,Longitude,Objetivo da Intervenção,Sonda,Terra ou Mar
0,1-BGM-6-ES,1-AJR-1-ES,ES-T-506,Espírito Santo,PA-1-BGM-6-ES_ES-T-506_R11,48610005466201372,BGM,18/08/2022,0,"\t-19:18:19,427","\t-39:50:00,328",TLD,Cowan-02,TERRA
1,1-BP-13-SPS,1BP13,BUMERANGUE,Santos,NaN,48610232917202233,BP Energy,25/05/2025,"2372,7","\t-26:29:34,530","\t-43:27:53,400",Perfuração,Valaris Renaissance,MAR
2,1-BRSA-1401DA-RJS,1-RJS-763DA,ÁGUA-MARIN,Campos,NaN,48610232915202244,Petrobras,11/07/2025,2601,"\t-22:51:52,333","\t-40:01:31,683",Perfuração,DEEPWATER AQUILA,MAR
3,1-GPK-4D-BA,1-PRC-1D-BA,REC-T-128,Recôncavo,TIRIBA,48610010821201597,Slim Drilling,12/12/2018,0,"\t-12:16:15,339","\t-38:19:48,945",Restauração,BCH Energy 5,TERRA
4,1-GREN-1D-BA,1-GE-0001-BA,REC-T-108,Recôncavo,PA-1GREN1DBA_REC-T-108,48610010812201504,Vultur Oil,21/12/2018,0,"\t-12:11:06,979","\t-38:10:58,999",TLD,GREAT - 128,TERRA
5,1-MPE-1-BA,1MPE1BA,BT-REC-3,Recôncavo,UIRAPURU,486100038992000,Petrosynergy,13/05/2003,0,"\t-12:19:32,320","\t-38:24:45,642",Restauração,SONDA PIONEIRA BRASIL,TERRA
6,3-AR-4-BA,3AR 0004 BA,AR,Recôncavo,ARAÇÁS,480000036319795,Petrobras,11/08/1965,NaN,"\t-12:08:22,096","\t-38:09:42,803",Abandono,SONDA CONVENCIONAL 25,TERRA
7,3-BRSA-1097-RN,3CS3RN,POT-T-609,Potiguar,MAÇARICO,48610001502200942,PetroRecôncavo,28/07/2012,0,"\t-05:20:13,457","\t-37:11:32,274",Completação,SONDA CONVENCIONAL 95,TERRA
8,3-BRSA-1177D-BA,3TQ210DBA,NaN,Recôncavo,TAQUIPE,480000037009714,Petrobras,21/05/2013,0,"\t-12:26:40,780","\t-38:27:55,500",Restauração,SONDA CONVENCIONAL 108,TERRA
9,3-BRSA-1320-RN,3ARG900RN,NaN,Potiguar,ALTO DO RODRIGUES,480000037849704,3R Potiguar,21/09/2015,0,"\t-05:18:32,467","\t-36:41:38,554",Completação,SONDA CONVENCIONAL 86,TERRA


In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

melhor_r2 = -np.inf
melhor_rmse = None
melhor_modelo = None
melhor_seed = None

for seed in range(5):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=seed
    )

    modelo = RandomForestRegressor(random_state=seed, n_estimators=200)
    modelo.fit(X_train, y_train)

    y_pred = modelo.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f"Tentativa {seed+1} → RMSE: {rmse:.2f} | R²: {r2*100:.2f}%")

    if r2 > melhor_r2:
        melhor_r2 = r2
        melhor_rmse = rmse
        melhor_modelo = modelo
        melhor_seed = seed

print("\n🏁 Melhor Resultado:")
print(f"Seed: {melhor_seed}")
print(f"RMSE: {melhor_rmse:.2f}")
print(f"R²: {melhor_r2*100:.2f}%")


Tentativa 1 → RMSE: 28.77 | R²: -51.44%
Tentativa 2 → RMSE: 25.06 | R²: -5.18%
Tentativa 3 → RMSE: 25.44 | R²: -110.91%
Tentativa 4 → RMSE: 21.75 | R²: 31.76%
Tentativa 5 → RMSE: 29.04 | R²: 23.11%

🏁 Melhor Resultado:
Seed: 3
RMSE: 21.75
R²: 31.76%


O treinamento foi realizado para prever a produtividade inicial (bpd) de poços exploratórios com base em variáveis geológicas e de reservatório. Após múltiplas tentativas com diferentes sementes, o melhor modelo — um Random Forest Regressor — alcançou um RMSE de 21.75 e um R² de 31.76%, mostrando bom desempenho na captura das variações reais dos dados simulados.